# 3 Notebooks :
> ## 1. Formating-Input : 
- data for each line (MAC adress, Access Point, Type of building, Action ...) in Dataframe 

> ## 2. Neo4j-Database 
- Fill Database (create nodes and relationships)
- Queries DB and fill "global results" Dataframe

> ## 3. Clustering and answer problematic (Who are students and teachers)

In [1]:
import pandas as pd

### Extract data from 378 Mo logs 

- for each line : MAC adress, Access Point, Type of building, Action ... in Dataframe

In [2]:
# Complete list of outputs
outputs = "./drive-download-20170906T133833Z-001/ls.output"

header = ["id", "month", "day", "hour", "AccessPoint1", "AccessPoint2", "status", "Message" ]

data = []
with open(outputs) as outputs:
    for myfileName in outputs :
        myfile =  "./drive-download-20170906T133833Z-001/"+myfileName.strip('\n')
        
        with open(myfile) as file: 
            for line in file :
                data.append(line.split(" ", 7))


In [3]:
df = pd.DataFrame(data, columns=header)

In [4]:
df.head()

,id,month,day,hour,AccessPoint1,AccessPoint2,status,Message
0,986990216,Apr,11,07:56:56,AdmBldg19AP3,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n
1,986990247,Apr,11,07:57:27,AdmBldg19AP3,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n
2,986990247,Apr,11,07:57:27,AdmBldg19AP3,AdmBldg19AP3,(Info):,Station 0040961e58be Associated\n
3,986990293,Apr,11,07:58:13,AdmBldg19AP3,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n
4,986990364,Apr,11,07:59:24,AdmBldg19AP3,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n


In [5]:
actions = ["authenticated", "reassociated", "disassociating", "associated",  "deauthenticating", "started", "stopped", "bound", "roamed", "updated"]

In [6]:
# Add action column
act = []
for mess in df.Message :
    caught = False
    for action in actions :
        if action in mess.lower() and caught == False:
            caught = True
            act.append(action)
    if caught == False :
        act.append("no action")    
df["action"]=act

In [7]:
# Drop redundant Access Point column
df = df.rename(index=str, columns={"AccessPoint1": "AccessPoint"})
df = df.drop('AccessPoint2', 1)

In [8]:
# Add Type building column
dico = {"Aca": "Academic", "Adm":"Admin", "Lib":"Library", "Res":"Residential", "Soc":"Social"}
buildingType = []
for AP in df.AccessPoint :
    try:
        buildingType.append(dico.get(AP[:3]))
    except TypeError:
        print AP[:3]   
df["Building Type"] = buildingType 

In [9]:
df.head()

,id,month,day,hour,AccessPoint,status,Message,action,Building Type
0,986990216,Apr,11,07:56:56,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin
1,986990247,Apr,11,07:57:27,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin
2,986990247,Apr,11,07:57:27,AdmBldg19AP3,(Info):,Station 0040961e58be Associated\n,associated,Admin
3,986990293,Apr,11,07:58:13,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin
4,986990364,Apr,11,07:59:24,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin


In [10]:
# Extract MAC addresses from support file 
addresses = []
with open( "./drive-download-20170906T133833Z-001/wirelessMACs") as MACaddressesFile:
    addresses = [address.strip('\n') for address in MACaddressesFile]


In [11]:
addresses[-1]

'ffffff31d6c0'

In [12]:
# Add MAC address column
addMac=[]
for message in df.Message :
    found = False
    for address in addresses :
        if (address in message):
            addMac.append(address)
            found = True
    if (found == False) :
        addMac.append("nothing") 
df["MAC address"] = addMac 

In [13]:
df.head()

,id,month,day,hour,AccessPoint,status,Message,action,Building Type,MAC address
0,986990216,Apr,11,07:56:56,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin,0040961e58be
1,986990247,Apr,11,07:57:27,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin,0040961e58be
2,986990247,Apr,11,07:57:27,AdmBldg19AP3,(Info):,Station 0040961e58be Associated\n,associated,Admin,0040961e58be
3,986990293,Apr,11,07:58:13,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin,0040961e58be
4,986990364,Apr,11,07:59:24,AdmBldg19AP3,(Info):,Station 0040961e58be Authenticated\n,authenticated,Admin,0040961e58be


In [14]:
df.to_csv("./data.csv")